<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9D%B4%EC%9A%A9%EB%AF%BC_%EB%B2%88%EC%97%AD_zero_shot_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#zero-shot classification

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/프로젝트/final project/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'cuda'

In [2]:
df = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_0905_clean.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24998 entries, 0 to 24997
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   review  24998 non-null  object 
 1   reply   24998 non-null  object 
 2   star    24998 non-null  int64  
 3   star_t  24998 non-null  int64  
 4   star_q  24998 non-null  int64  
 5   star_d  24998 non-null  float64
 6   food    23986 non-null  object 
 7   store   24998 non-null  object 
dtypes: float64(1), int64(3), object(4)
memory usage: 1.5+ MB


In [3]:
df.isnull().sum()

review       0
reply        0
star         0
star_t       0
star_q       0
star_d       0
food      1012
store        0
dtype: int64

# zero-shot classification

In [4]:
!pip install transformers

In [5]:
from transformers import pipeline

# Zero-shot classification pipeline 생성
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")



In [6]:
#런타임 재시작 필요
!pip install sentencepiece
!pip install accelerate -U

In [7]:
from transformers import MarianMTModel, MarianTokenizer, pipeline

# 번역 모델 및 토크나이저 초기화
model_name = 'Helsinki-NLP/opus-mt-ko-en'
model = MarianMTModel.from_pretrained(model_name).to('cuda')  # 모델을 GPU로 이동
tokenizer = MarianTokenizer.from_pretrained(model_name)

# 한글 리뷰를 영어로 번역하는 함수 (배치 처리)
def batch_translate_to_english(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to('cuda')  # 입력 데이터를 GPU로 이동
    translated = model.generate(**inputs)
    return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

# Zero-shot classification pipeline 생성
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)  # GPU를 사용하기 위해 device=0 설정


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [8]:
from tqdm import tqdm

# 주제 레이블 정의
labels = ['Taste', 'Price', 'Delivery']

# 리뷰 데이터에서 번역 및 주제 판단
results = []
batch_size = 16

# tqdm을 사용하여 진행률 바 추가
for i in tqdm(range(0, len(df['review'][:1000]), batch_size)):
    reviews_batch = df['review'][i:i+batch_size].tolist()
    translated_reviews_batch = batch_translate_to_english(reviews_batch)
    for translated_review in translated_reviews_batch:
        result = classifier(translated_review, labels)
        results.append(result['labels'][0])


results = pd.DataFrame(results, columns=["label"])

results.to_csv("results.csv", index=False)

  1%|          | 2/250 [00:07<13:12,  3.20s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 250/250 [06:18<00:00,  1.52s/it]


In [27]:
from tqdm import tqdm

# 주제 레이블 정의
labels = ['Taste', 'Price', 'Delivery']

# 리뷰 데이터에서 번역 및 주제 판단
results = []
batch_size = 16

# tqdm을 사용하여 진행률 바 추가
for i in tqdm(range(0, len(df['review'][:1000]), batch_size)):  # df['review'][:1000] 범위만 처리
    reviews_batch = df['review'][i:i+batch_size].tolist()
    translated_reviews_batch = batch_translate_to_english(reviews_batch)
    for translated_review in translated_reviews_batch:
        result = classifier(translated_review, labels)
        results.append(result['labels'][0])

translated_df = pd.DataFrame(translated_reviews, columns=["translated_review"])


100%|██████████| 63/63 [06:32<00:00,  6.23s/it]


In [9]:
# 결과 파일 로드
results = pd.read_csv("results.csv")
results = results["label"].tolist()

In [10]:
from collections import Counter

label_counts = Counter(results)
print(label_counts)

Counter({'Taste': 816, 'Delivery': 103, 'Price': 81})


In [11]:
import random
random.seed(42)
num_samples = 5


for label in labels:
    print(f"### {label} ###")

    # 해당 레이블 리뷰 인덱스 추출
    indices = [i for i, x in enumerate(results) if x == label]

    # 인덱스 중에서 무작위로 샘플링
    sampled_indices = random.sample(indices, min(num_samples, len(indices)))

    for idx in sampled_indices:
        print(f"- {df['review'].iloc[idx]}")
    print("\n")


### Taste ###
- 양도 많고 맛있어요
- 맛있어요 또시킬게용
- 양도 많고 맛있어요 
- 사진 저장이 안돼고 날아가버렸네요 재주문의사도 5점일만큼 구성도 좋고 맛있어요 
- 타브랜드와비교해서 진짜너무너무맛있어요 다만 양은타브랜드에비해 좀 적어요 


### Price ###
- 배송 엄청 빨라요 
- 국밥은 항상 여기 
- 커피는 워커스입니다 좋아요
- 떡깨비 예전 동네에서 못 찾았다가 이사 와서 시켰는데 똑같이 맛있네요 치즈 양이 어느 떡볶이 브랜드보다 많아요 게다가 맛있습니다 다음에도 시킬 거예요 
- 핫시리즈들 정말 강추합니다 


### Delivery ###
- 한그릇 배달도 좋고 너무 국물이 맛있어요
- 배달팁도 없고 맛있어요
- 주문할때마다 너무너무 만족스럽습니다 맛있는 치킨 보내주셔서 감사합니다
- 배달도 빠르고 역시 맛잇어요 치즈 진짜 엄청 많이주셧습니다 야채튀김은 진짜 필수에요 자주 시켜먹을겁니다 서비스 많이 주셔서 감사합니다 
- 역시 치킨은 당치땡 정말 맛있어요 양도많고 배달도 예상시간보다 빨리 왔어요 다음에 또 주문할께요 




# 번역 리뷰 출력
# (적절한 번역 불가)

In [31]:
import random

num_samples = 5

for label in labels:
    print(f"### {label} ###")

    indices = [i for i, x in enumerate(results) if x == label]
    sampled_indices = random.sample(indices, min(num_samples, len(indices)))

    for idx in sampled_indices:
        print(f"- {translated_df['translated_review'].iloc[idx]}")
    print("\n")


### Taste ###
- That's good. That's good.
- I'm John Mattin.
- It was fine.
- I ordered it for the first time, but it's a very generous meal. Thank you.
- It's good, it's good, it's good, it's good, it's good, it's not smelly, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it's good, it'

# 데이터 분리

In [13]:
from sklearn.model_selection import train_test_split

# results 리스트를 첫 1000개만 사용 - > 배치 처리로 인해 1008로 리뷰가 8개 초과됨
results = results[:1000]

# 데이터 분리 (학습 데이터와 검증 데이터)
train_texts, val_texts, train_labels, val_labels = train_test_split(df['review'][:1000], results, test_size=0.2)


# 제로샷 튜닝

 - 분류 후처리

In [ ]:
def post_process_label(review, predicted_label):
    taste_keywords = ["잘 먹었습니다", "맛있게 먹었습니다", "맛있게 잘먹었습니다"]
    for keyword in taste_keywords:
        if keyword in review:
            return 'Taste'
    return predicted_label

# 학습 데이터에 대해서만 분류 후처리 적용
processed_train_labels = [post_process_label(review, label) for review, label in zip(train_texts, train_labels)]


 - fine-tuning
 -  'facebook/bart-large-mnli' 모델은 3개의 클래스를 예측하도록 학습되었으므로 3개로 조정.

In [ ]:
from transformers import BartTokenizer, BartForSequenceClassification, Trainer, TrainingArguments
from transformers import BartConfig
import torch

# 1. 데이터 준비
labels_dict = {'Taste': 0, 'Price': 1, 'Delivery': 2}

# processed_train_labels를 사용하여 학습 데이터의 레이블을 숫자 형태로 변환
train_labels = [labels_dict[label] for label in processed_train_labels]

#모델 토크나이저 로드
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-mnli')

# 학습, 검증 데이터 인코딩
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=512)

class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 학습, 검증 데이터셋 객체 생성
train_dataset = ReviewDataset(train_encodings, train_labels)
val_dataset = ReviewDataset(val_encodings, [labels_dict[label] for label in val_labels])



# 2. 모델 및 토크나이저 초기화
# 레이블을 모델과 맞게 조정하는 과정. 레이블이 추가된다면 이 코드 사용
config = BartConfig.from_pretrained('facebook/bart-large-mnli')
config.num_labels = len(labels_dict)

# 수정된 설정 사용하여 모델 초기화
model = BartForSequenceClassification.from_pretrained('facebook/bart-large-mnli', config=config)

# # 평가 지표 계산 함수 정의
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return {"accuracy": accuracy_score(labels, predictions)}

# 3. 모델 학습
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,              # 전체 데이터셋에 대한 학습 에폭
    evaluation_strategy="epoch",     # 에폭마다 평가 수행
    save_strategy="epoch",           # 에폭 모델을 저장
    logging_dir='./logs',            # 로그가 저장될 디렉토리 경로
    logging_steps=10,                # 10 스텝마다 로깅 정보를 출력
    do_train=True,                   # 학습을 수행할 것인지 지정
    do_eval=True,                    # 평가를 수행할 것인지 지정
    output_dir='./results',          # 모델 및 결과가 저장될 디렉토리 경로
    overwrite_output_dir=True,       # 지정된 출력 디렉토리가 이미 있으면 덮어쓰기
    logging_first_step=False,        # 첫 번째 스텝에서 로깅을 시작할 것인지 지정


    load_best_model_at_end=True      # 학습이 끝난 후 최적의 모델을 불러올 것인지 지정
)

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
    # compute_metrics=compute_metrics  # 평가 지표 계산 함수를 지정
)

# 모델 학습 시
trainer.train()


OutOfMemoryError: ignored

In [ ]:
from transformers import BartTokenizer, BartForSequenceClassification, Trainer, TrainingArguments
from transformers import BartConfig
import torch

# 1. 데이터 준비
labels_dict = {'Taste': 0, 'Price': 1, 'Delivery': 2}

# processed_train_labels를 사용하여 학습 데이터의 레이블을 숫자 형태로 변환
train_labels = [labels_dict[label] for label in processed_train_labels]

#모델 토크나이저 로드
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-mnli')

# 학습, 검증 데이터 인코딩
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=512)

class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 학습, 검증 데이터셋 객체 생성
train_dataset = ReviewDataset(train_encodings, train_labels)
val_dataset = ReviewDataset(val_encodings, [labels_dict[label] for label in val_labels])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)


# 2. 모델 및 토크나이저 초기화
# 레이블을 모델과 맞게 조정하는 과정. 레이블이 추가된다면 이 코드 사용
config = BartConfig.from_pretrained('facebook/bart-large-mnli')
config.num_labels = len(labels_dict)

# 수정된 설정 사용하여 모델 초기화
model = BartForSequenceClassification.from_pretrained('facebook/bart-large-mnli', config=config)

# 커스텀 학습 및 평가 루프
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# # 평가 지표 계산 함수 정의
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return {"accuracy": accuracy_score(labels, predictions)}

# 학습 및 검증 루프
num_epochs = 3
for epoch in range(num_epochs):
    # 학습 루프
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        inputs = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch: {epoch+1}, Training Loss: {total_loss / len(train_loader)}")

    # 검증 루프
    model.eval()
    total_eval_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs = {key: val.to(device) for key, val in batch.items()}
            outputs = model(**inputs)
            total_eval_loss += outputs.loss.item()

    print(f"Epoch: {epoch+1}, Validation Loss: {total_eval_loss / len(val_loader)}")